In [4]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np




In [9]:
price_response_df = pd.read_csv('/Users/youziya/Library/CloudStorage/OneDrive-YorkUniversity/OMIS 6000 Models and Applications in Operational Reseach/Assignment 2/price_response.csv')

a)

In [27]:
from scipy.optimize import minimize
import numpy as np

# Coefficients for the Basic and Advanced versions
a1, b1 = 35234.545786, 45.896450
a2, b2 = 37790.240832, 8.227794

# Define the objective function to be minimized (negative revenue since we're using a minimization function)
def objective(p):
    p1, p2 = p
    return -((p1 * (a1 - b1 * p1)) + (p2 * (a2 - b2 * p2)))

# Initial guess (starting prices)
p_initial = [100, 150]  # Arbitrary initial prices for Basic and Advanced versions

# Constraints
# p2 >= p1 (Advanced price should be higher than Basic)
# Prices p1, p2 >= 0 are inherently considered by the bounds parameter
constraints = [{'type': 'ineq', 'fun': lambda p: p[1] - p[0]}]  # Advanced version price - Basic version price >= 0

# Bounds for p1 and p2 (assuming a realistic minimum price of 0 and a maximum price that could be reasonably set for laptops)
bounds = [(0, None), (0, None)]

# Solve the optimization problem
result = minimize(objective, p_initial, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimal prices
p_optimal = result.x

p_optimal, -result.fun  # Display optimal prices and maximum revenue


(array([ 383.8440553 , 2296.50832013]), 50154984.20934893)

b)

c)

In [25]:
from gurobipy import Model, GRB

# Create a new Gurobi model
model = Model("OptimalPricingStrategy")

# Assuming hypothetical linear price-response functions for simplicity
# a_ij and b_ij could be specific to each product version in each product line
# For example purposes, these are generic coefficients
a = {('Basic', 'EvoTech'): 1000, ('Advanced', 'EvoTech'): 1500, ('Premium', 'EvoTech'): 2000,
     ('Basic', 'InfiniteEdge'): 1100, ('Advanced', 'InfiniteEdge'): 1600, ('Premium', 'InfiniteEdge'): 2100,
     ('Basic', 'FusionBook'): 1200, ('Advanced', 'FusionBook'): 1700, ('Premium', 'FusionBook'): 2200}
b = {version: 10 for version in a}  # Assuming a uniform sensitivity for simplicity

# Decision variables for prices
prices = model.addVars(a.keys(), name="P", lb=0, vtype=GRB.CONTINUOUS)

# Objective: Maximize total revenue
model.setObjective(sum((prices[version] * (a[version] - b[version] * prices[version])) for version in a), GRB.MAXIMIZE)

# Constraints: Prices within each product line increase from Basic to Advanced to Premium
model.addConstr(prices[('Basic', 'EvoTech')] <= prices[('Advanced', 'EvoTech')], "EvoTech_Basic_Advanced")
model.addConstr(prices[('Advanced', 'EvoTech')] <= prices[('Premium', 'EvoTech')], "EvoTech_Advanced_Premium")

model.addConstr(prices[('Basic', 'InfiniteEdge')] <= prices[('Advanced', 'InfiniteEdge')], "InfiniteEdge_Basic_Advanced")
model.addConstr(prices[('Advanced', 'InfiniteEdge')] <= prices[('Premium', 'InfiniteEdge')], "InfiniteEdge_Advanced_Premium")

model.addConstr(prices[('Basic', 'FusionBook')] <= prices[('Advanced', 'FusionBook')], "FusionBook_Basic_Advanced")
model.addConstr(prices[('Advanced', 'FusionBook')] <= prices[('Premium', 'FusionBook')], "FusionBook_Advanced_Premium")

# Solve the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for version in a:
        var = prices[version]  # Directly access the variable from the dictionary
        print(f"{version[1]} {version[0]} Price: ${var.X:.2f}")
    print(f"Optimal Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 12 nonzeros
Model fingerprint: 0x91060efe
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+03]
  QObjective range [2e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.31200000e+08  3.60000000e+08  0.00e+00 0.00e+00  9.90e+0

d)

In [23]:
from gurobipy import Model, GRB

# Example data structures for intercepts and sensitivities
# Adjust with your actual data
intercepts = {(i, j): 10000 for i in range(3) for j in range(3)}  # Placeholder values
sensitivities = {(i, j): 50 for i in range(3) for j in range(3)}  # Placeholder values

# Initialize the model
model = Model("TechEssentials Pricing Optimization")

# Define decision variables for prices
prices = model.addVars(3, 3, lb=0, name="P", vtype=GRB.CONTINUOUS)

# Set the objective: Maximize total revenue
model.setObjective(sum(prices[i, j] * (intercepts[i, j] - sensitivities[i, j] * prices[i, j])
                       for i in range(3) for j in range(3)), GRB.MAXIMIZE)

# Constraint 1: Prices increase within each product line (Basic < Advanced < Premium)
for i in range(3):
    model.addConstr(prices[i, 0] <= prices[i, 1], f"Line_{i}_Basic_Advanced")
    model.addConstr(prices[i, 1] <= prices[i, 2], f"Line_{i}_Advanced_Premium")

# Constraint 2: Prices increase across product lines for the same version
# EvoTech Pro < InfiniteEdge Notebook < FusionBook Elite for each version
# Additional constraints for part (d) ensuring inter-line price increase for the same versions
for j in range(3):  # For each version (Basic, Advanced, Premium)
    model.addConstr(prices[(0, j)] <= prices[(1, j)], "InterLine_Basic_EvoTech_InfiniteEdge")
    model.addConstr(prices[(1, j)] <= prices[(2, j)], "InterLine_Advanced_InfiniteEdge_FusionBook")


# Solve the model
model.optimize()

# Output the optimal prices and the anticipated optimal revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices for Each Product Version in Each Product Line:")
    for i in range(3):
        for j in range(3):
            print(f"Product Line {i+1}, Version {j+1}: ${prices[i, j].X:.2f}")
    print(f"\nAnticipated Optimal Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x0fe8be80
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 1e+04]
  QObjective range [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.62000000e+09  1.80000000e+09  0.00e+00 0.00e+00  9.87e

In [26]:
from gurobipy import Model, GRB

# Initialize the Gurobi model
model = Model("OptimalPricingStrategyEnhanced")

# Define decision variables for prices, considering three product lines and three versions each
# Using a simplified structure for indices: (product_line, version)
# Product lines: 0 = EvoTech Pro, 1 = InfiniteEdge Notebook, 2 = FusionBook Elite
# Versions: 0 = Basic, 1 = Advanced, 2 = Premium
prices = model.addVars([(line, version) for line in range(3) for version in range(3)], name="price", lb=0, vtype=GRB.CONTINUOUS)

# Example intercepts and sensitivities for the price-demand function, to be customized as per actual data
intercepts = {(line, version): 1000 - 10*line + 100*version for line in range(3) for version in range(3)}
sensitivities = {(line, version): 2 + 0.5*version for line in range(3) for version in range(3)}

# Set the objective: Maximize total revenue
model.setObjective(
    sum(prices[(line, version)] * (intercepts[(line, version)] - sensitivities[(line, version)] * prices[(line, version)])
        for line in range(3) for version in range(3)), GRB.MAXIMIZE)

# Add constraints for increasing prices within each product line
for line in range(3):
    for version in range(2):  # Only need to compare Basic to Advanced, and Advanced to Premium
        model.addConstr(prices[(line, version)] <= prices[(line, version+1)], f"WithinLine_{line}_{version}")

# Add constraints for increasing prices across product lines for the same version
for version in range(3):
    model.addConstr(prices[(0, version)] <= prices[(1, version)], f"AcrossLines_0_{version}")
    model.addConstr(prices[(1, version)] <= prices[(2, version)], f"AcrossLines_1_{version}")

# Solve the optimization problem
model.optimize()

# Output the optimal prices and anticipated revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices for Each Version in Each Product Line:")
    for line in range(3):
        for version in range(3):
            print(f"Product Line {line+1}, Version {version+1}: ${prices[(line, version)].X:.2f}")
    print(f"Optimal Total Revenue: ${model.objVal:.2f}")
else:
    print("No optimal solution was found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D60)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 12 rows, 9 columns and 24 nonzeros
Model fingerprint: 0x86c15e53
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+03]
  QObjective range [4e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.20760000e+08  3.60000000e+08  0.00e+00 0.00e+00  9.77e

e)In practice, the most sensible prices to implement would ideally follow the logic of (d), where there's a clear differentiation in pricing both within each product line and across product lines for the same versions, reflecting both product value and strategic market positioning. However, it's critical to ensure that the optimization model correctly implements these constraints and that the output accurately reflects this logic. The reported identical prices for all versions in the FusionBook line under (d) suggest a need to revisit the model or output interpretation to ensure it aligns with practical business strategies and product differentiation goals.

f) the consumer behaviour might change nd hard for us to predict. there are also external factors and economic condition.